In [5]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
%%capture
%run "22. Análisis post hoc de cambios promedios.ipynb"

In [7]:
# Definir las columnas de cambios de opinión por ítem individual.
Columnas_Cambios = Columnas_CO_Pro + Columnas_CO_Con 
                    # + Columnas_CT_Pro + Columnas_CT_Con

In [ ]:
Diccionario_P_Valores = {'Generales':{},
                         'Ballotage':{}}

In [ ]:
for Nombre_df, df in dfs_Finales.items():

    print("\n" + "="*50)
    print(f"Comparaciones estadísticas para el conjunto de datos: {Nombre_df}")
    print("\n" + "="*50)

    # Crear un DataFrame con las columnas necesarias.
    df_Cambios_Item = df[['ID', 'Categoria_PASO_2023'] + Columnas_CO_Pro + Columnas_CO_Con 
        # + Columnas_CT_Pro + Columnas_CT_Con
        ]
    
    # Filtrar filas con categorías válidas y sin valores nulos en las columnas de interés.
    Categorias_Validas = [
        'Left_Wing', 
        'Progressivism', 
        'Centre',
        'Moderate_Right_A', 
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]

    # Filtrar solo las categorías válidas
    df_Cambios_Item = df_Cambios_Item[df_Cambios_Item['Categoria_PASO_2023'].isin(Categorias_Validas)]


    from scipy.stats import kruskal

    # Crear diccionario para almacenar resultados.
    Resultados_Kruskal = {}

    # Agrupar por categoría para cada columna y aplicar Kruskal-Wallis.
    for Nombre_Columna in Columnas_Cambios:
        Grupos = [
            df_Cambios_Item[df_Cambios_Item['Categoria_PASO_2023'] == Categoria][Nombre_Columna].dropna()
            for Categoria in Categorias_Validas
        ]

        # Verificar que todos los grupos tengan al menos dos datos.
        if all(len(Grupo) > 1 for Grupo in Grupos):
            Estadistico, P_Valor = kruskal(*Grupos)
            Resultados_Kruskal[Nombre_Columna] = {
                'Estadístico': Estadistico,
                'Valor_p': P_Valor
            }

        # Convertir los resultados a DataFrame.
        Tabla_Resultados_Kruskal = pd.DataFrame.from_dict(
            Resultados_Kruskal, orient='index'
        ).reset_index().rename(columns={
            'index': 'Variable',
            'Estadístico': 'Estadistico_Kruskal',
            'Valor_p': 'Valor_p'
        })
        # Ordenar por p-valor ascendente.
        Tabla_Resultados_Kruskal = Tabla_Resultados_Kruskal.sort_values(
            by='Valor_p'
        ).reset_index(drop=True)
        Tabla_Valor_P_Menor_005 = Tabla_Resultados_Kruskal[
            Tabla_Resultados_Kruskal['Valor_p'] < 0.05
        ]
    
    # Mostrar la tabla de resultados.
    print(Tabla_Resultados_Kruskal)

    # Extraer las variables significativas.
    Variables_Significativas_Items = Tabla_Valor_P_Menor_005['Variable'].tolist()

    # Guardar las variables significativas en el diccionario global.
    Todas_Variables_Significativas['Cambios_Por_Item_Significativos'][Nombre_df] = Variables_Significativas_Items

    # Mostrar resultados.
    print(f"Variables significativas para {Nombre_df}: {len(Variables_Significativas_Items)} variables")
    print(Variables_Significativas_Items)

    # Recolectar p-valores en el diccionario.
    for _, Fila in Tabla_Resultados_Kruskal.iterrows():
        Variable = Fila['Variable']
        P_Valor = Fila['Valor_p']
        Diccionario_P_Valores[Nombre_df][Variable] = P_Valor